# How to Train Your Own Cone Detection Networks

![](https://user-images.githubusercontent.com/22118253/70957091-fe06a480-2042-11ea-8c06-0fcc549fc19a.png)

In this notebook, we will demonstrate 
- how to train your own YOLOv3-based traffic cone detection network and do inference on a video.

**[Accurate Low Latency Visual Perception for Autonomous Racing: Challenges Mechanisms and Practical Solutions](https://github.com/mit-han-lab/once-for-all)** is an accurate low latency visual perception system introduced by Kieran Strobel, Sibo Zhu, Raphael Chang, and Skanda Koppula.

## 1. Preparation
Let's first install all the required packages:

In [1]:
! sudo apt install unzip
print('Installing PyTorch...')
! pip3 install torch 
print('Installing torchvision...')
! pip3 install torchvision 
print('Installing numpy...')
! pip3 install numpy 
# tqdm is a package for displaying a progress bar.
print('Installing tqdm (progress bar) ...')
! pip3 install tqdm 
print('Installing matplotlib...')
! pip3 install matplotlib 
print('Installing all the other required packages once for all')
! sudo python3 setup.py install
print('Installing video editor')
! sudo apt install ffmpeg -y 

Reading package lists... Done
Building dependency tree       
Reading state information... Done
unzip is already the newest version (6.0-21ubuntu1).
The following packages were automatically installed and are no longer required:
  cuda-10-1 cuda-command-line-tools-10-1 cuda-compiler-10-1 cuda-cudart-10-1
  cuda-cudart-dev-10-1 cuda-cufft-10-1 cuda-cufft-dev-10-1 cuda-cuobjdump-10-1
  cuda-cupti-10-1 cuda-curand-10-1 cuda-curand-dev-10-1 cuda-cusolver-10-1
  cuda-cusolver-dev-10-1 cuda-cusparse-10-1 cuda-cusparse-dev-10-1
  cuda-demo-suite-10-1 cuda-documentation-10-1 cuda-driver-dev-10-1
  cuda-gdb-10-1 cuda-gpu-library-advisor-10-1 cuda-libraries-10-1
  cuda-libraries-dev-10-1 cuda-license-10-1 cuda-memcheck-10-1
  cuda-misc-headers-10-1 cuda-npp-10-1 cuda-npp-dev-10-1 cuda-nsight-10-1
  cuda-nsight-compute-10-1 cuda-nsight-systems-10-1 cuda-nvcc-10-1
  cuda-nvdisasm-10-1 cuda-nvgraph-10-1 cuda-nvgraph-dev-10-1 cuda-nvjpeg-10-1
  cuda-nvjpeg-dev-10-1 cuda-nvml-dev-10-1 cuda-nvprof-10-

Installing video editor
Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
The following packages were automatically installed and are no longer required:
  cuda-10-1 cuda-command-line-tools-10-1 cuda-compiler-10-1 cuda-cudart-10-1
  cuda-cudart-dev-10-1 cuda-cufft-10-1 cuda-cufft-dev-10-1 cuda-cuobjdump-10-1
  cuda-cupti-10-1 cuda-curand-10-1 cuda-curand-dev-10-1 cuda-cusolver-10-1
  cuda-cusolver-dev-10-1 cuda-cusparse-10-1 cuda-cusparse-dev-10-1
  cuda-demo-suite-10-1 cuda-documentation-10-1 cuda-driver-dev-10-1
  cuda-gdb-10-1 cuda-gpu-library-advisor-10-1 cuda-libraries-10-1
  cuda-libraries-dev-10-1 cuda-license-10-1 cuda-memcheck-10-1
  cuda-misc-headers-10-1 cuda-npp-10-1 cuda-npp-dev-10-1 cuda-nsight-10-1
  cuda-nsight-compute-10-1 cuda-nsight-systems-10-1 cuda-nvcc-10-1
  cuda-nvdisasm-10-1 cuda-nvgraph-10-1 cuda-nvgraph-dev-10-1 cuda-nvjpeg-10-1
  cuda-nvjpeg-dev-10-1 cuda

Let' s clone our repo first...

In [ ]:
! git clone https://github.com/cv-core/MIT-Driverless-CV-TrainingInfra.git

! mv MIT-Driverless-CV-TrainingInfra/CVC-YOLOv3/*

Before we start training, let's download the Cone Detection dataset and the corresponding label and intial training weights. 

In [2]:
print("Downloading Training Dataset")
! wget https://storage.googleapis.com/mit-driverless-open-source/YOLO_Dataset.zip
! unzip YOLO_Dataset.zip
! mv YOLO_Dataset dataset/ && rm YOLO_Dataset.zip
print("Downloading YOLOv3 Sample Weights")
! wget https://storage.googleapis.com/mit-driverless-open-source/yolov3-training/sample-yolov3.weights 
print("Downloading Training and Validation Label")
! cd dataset/ && wget https://storage.googleapis.com/mit-driverless-open-source/yolov3-training/all.csv && cd ..
! cd dataset/ && wget https://storage.googleapis.com/mit-driverless-open-source/yolov3-training/train_mini_yolo.csv && mv train_mini_yolo.csv train.csv && cd ..
! cd dataset/ && wget https://storage.googleapis.com/mit-driverless-open-source/yolov3-training/validate_mini_yolo.csv && mv validate_mini_yolo.csv validate.csv && cd ..

--2020-08-30 00:10:06--  https://storage.googleapis.com/mit-driverless-open-source/YOLO_Dataset.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.214.128, 172.217.212.128, 173.194.197.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.214.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1875926369 (1.7G) [application/zip]
Saving to: ‘YOLO_Dataset.zip’

YOLO_Dataset.zip    100%[===================>]   1.75G   171MB/s    in 11s     

2020-08-30 00:10:17 (159 MB/s) - ‘YOLO_Dataset.zip’ saved [1875926369/1875926369]

Archive:  YOLO_Dataset.zip
   creating: YOLO_Dataset/
  inflating: YOLO_Dataset/vid_100_frame_131.jpg  
  inflating: YOLO_Dataset/vid_37_frame_135.jpg  
  inflating: YOLO_Dataset/vid_6_frame_2210.jpg  
  inflating: YOLO_Dataset/vid_204_frame_12.jpg  
  inflating: YOLO_Dataset/vid_42_frame_209.jpg  
  inflating: YOLO_Dataset/vid_121_frame_152.jpg  
  inflating: YOLO_Dataset/vid_96_frame_49.jpg  
  i

  inflating: YOLO_Dataset/vid_18_frame_1410.jpg  
  inflating: YOLO_Dataset/vid_5_frame_1799.jpg  
  inflating: YOLO_Dataset/vid_5_frame_1654.jpg  
  inflating: YOLO_Dataset/vid_125_frame_2740.jpg  
  inflating: YOLO_Dataset/vid_85_frame_49.jpg  
  inflating: YOLO_Dataset/vid_3_frame_15904.jpg  
  inflating: YOLO_Dataset/vid_102_frame_140.jpg  
  inflating: YOLO_Dataset/vid_41_frame_190.jpg  
  inflating: YOLO_Dataset/vid_41_frame_804.jpg  
  inflating: YOLO_Dataset/vid_31_frame_1812.jpg  
  inflating: YOLO_Dataset/vid_41_frame_372.jpg  
  inflating: YOLO_Dataset/vid_2_frame_27256.jpg  
  inflating: YOLO_Dataset/vid_40_frame_950.jpg  
  inflating: YOLO_Dataset/vid_31_frame_1605.jpg  
  inflating: YOLO_Dataset/vid_59_frame_185.jpg  
  inflating: YOLO_Dataset/vid_31_frame_2587.jpg  
  inflating: YOLO_Dataset/vid_36_frame_214.jpg  
  inflating: YOLO_Dataset/vid_64_frame_483.jpg  
  inflating: YOLO_Dataset/vid_38_frame_328.jpg  
  inflating: YOLO_Dataset/vid_5_frame_1868.jpg  
  inflating:

  inflating: YOLO_Dataset/vid_30_frame_1863.jpg  
  inflating: YOLO_Dataset/vid_96_frame_64.jpg  
  inflating: YOLO_Dataset/vid_30_frame_1807.jpg  
  inflating: YOLO_Dataset/vid_31_frame_3136.jpg  
  inflating: YOLO_Dataset/vid_71_frame_332.jpg  
  inflating: YOLO_Dataset/vid_75_frame_266.jpg  
  inflating: YOLO_Dataset/vid_37_frame_252.jpg  
  inflating: YOLO_Dataset/vid_123_frame_8713.jpg  
  inflating: YOLO_Dataset/vid_3_frame_11517.jpg  
  inflating: YOLO_Dataset/vid_93_frame_72.jpg  
  inflating: YOLO_Dataset/vid_18_frame_1347.jpg  
  inflating: YOLO_Dataset/vid_71_frame_96.jpg  
  inflating: YOLO_Dataset/vid_31_frame_3192.jpg  
  inflating: YOLO_Dataset/vid_5_frame_2147.jpg  
  inflating: YOLO_Dataset/vid_31_frame_1824.jpg  
  inflating: YOLO_Dataset/vid_86_frame_67.jpg  
  inflating: YOLO_Dataset/vid_38_frame_260.jpg  
  inflating: YOLO_Dataset/vid_40_frame_622.jpg  
  inflating: YOLO_Dataset/vid_31_frame_2637.jpg  
  inflating: YOLO_Dataset/vid_31_frame_1710.jpg  
  inflating: 

  inflating: YOLO_Dataset/vid_18_frame_1278.jpg  
  inflating: YOLO_Dataset/vid_3_frame_31697.jpg  
  inflating: YOLO_Dataset/vid_31_frame_2156.jpg  
  inflating: YOLO_Dataset/vid_83_frame_43.jpg  
  inflating: YOLO_Dataset/vid_76_frame_372.jpg  
  inflating: YOLO_Dataset/vid_64_frame_78.jpg  
  inflating: YOLO_Dataset/vid_45_frame_882.jpg  
  inflating: YOLO_Dataset/vid_28_frame_1392.jpg  
  inflating: YOLO_Dataset/vid_42_frame_288.jpg  
  inflating: YOLO_Dataset/vid_41_frame_886.jpg  
  inflating: YOLO_Dataset/vid_98_frame_40.jpg  
  inflating: YOLO_Dataset/vid_66_frame_154.jpg  
  inflating: YOLO_Dataset/vid_18_frame_2132.jpg  
  inflating: YOLO_Dataset/vid_31_frame_3128.jpg  
  inflating: YOLO_Dataset/vid_75_frame_311.jpg  
  inflating: YOLO_Dataset/vid_31_frame_1918.jpg  
  inflating: YOLO_Dataset/vid_31_frame_1890.jpg  
  inflating: YOLO_Dataset/vid_125_frame_3901.jpg  
  inflating: YOLO_Dataset/vid_28_frame_1112.jpg  
  inflating: YOLO_Dataset/vid_102_frame_60.jpg  
  inflating:

  inflating: YOLO_Dataset/vid_38_frame_191.jpg  
  inflating: YOLO_Dataset/vid_38_frame_177.jpg  
  inflating: YOLO_Dataset/vid_121_frame_6453.jpg  
  inflating: YOLO_Dataset/vid_5_frame_1950.jpg  
  inflating: YOLO_Dataset/vid_36_frame_252.jpg  
  inflating: YOLO_Dataset/vid_40_frame_1370.jpg  
  inflating: YOLO_Dataset/vid_41_frame_656.jpg  
  inflating: YOLO_Dataset/vid_42_frame_743.jpg  
  inflating: YOLO_Dataset/vid_70_frame_107.jpg  
  inflating: YOLO_Dataset/vid_53_frame_13.jpg  
  inflating: YOLO_Dataset/vid_121_frame_5041.jpg  
  inflating: YOLO_Dataset/vid_120_frame_2866.jpg  
  inflating: YOLO_Dataset/vid_2_frame_5291.jpg  
  inflating: YOLO_Dataset/vid_91_frame_296.jpg  
  inflating: YOLO_Dataset/vid_31_frame_2343.jpg  
  inflating: YOLO_Dataset/vid_203_frame_33.jpg  
  inflating: YOLO_Dataset/vid_68_frame_229.jpg  
  inflating: YOLO_Dataset/vid_65_frame_120.jpg  
  inflating: YOLO_Dataset/vid_71_frame_248.jpg  
  inflating: YOLO_Dataset/vid_3_frame_4019.jpg  
  inflating: 

...

  inflating: YOLO_Dataset/vid_31_frame_3182.jpg  
  inflating: YOLO_Dataset/vid_18_frame_1970.jpg  
  inflating: YOLO_Dataset/vid_28_frame_1694.jpg  
  inflating: YOLO_Dataset/vid_38_frame_740.jpg  
  inflating: YOLO_Dataset/vid_3_frame_9144.jpg  
  inflating: YOLO_Dataset/vid_122_frame_3077.jpg  
  inflating: YOLO_Dataset/vid_31_frame_3181.jpg  
  inflating: YOLO_Dataset/vid_101_frame_115.jpg  
  inflating: YOLO_Dataset/vid_40_frame_1373.jpg  
  inflating: YOLO_Dataset/vid_109_frame_163.jpg  
  inflating: YOLO_Dataset/vid_31_frame_1855.jpg  
  inflating: YOLO_Dataset/vid_31_frame_2643.jpg  
  inflating: YOLO_Dataset/vid_6_frame_41028.jpg  
  inflating: YOLO_Dataset/vid_2_frame_1815.jpg  
  inflating: YOLO_Dataset/vid_78_frame_246.jpg  
  inflating: YOLO_Dataset/vid_53_frame_155.jpg  
  inflating: YOLO_Dataset/vid_38_frame_876.jpg  
  inflating: YOLO_Dataset/vid_31_frame_3198.jpg  
  inflating: YOLO_Dataset/vid_31_frame_2582.jpg  
  inflating: YOLO_Dataset/vid_45_frame_547.jpg  
  infl

  inflating: YOLO_Dataset/vid_40_frame_705.jpg  
  inflating: YOLO_Dataset/vid_37_frame_436.jpg  
  inflating: YOLO_Dataset/vid_28_frame_1055.jpg  
  inflating: YOLO_Dataset/vid_77_frame_48.jpg  
  inflating: YOLO_Dataset/vid_42_frame_402.jpg  
  inflating: YOLO_Dataset/vid_40_frame_1146.jpg  
  inflating: YOLO_Dataset/vid_108_frame_153.jpg  
  inflating: YOLO_Dataset/vid_31_frame_1524.jpg  
  inflating: YOLO_Dataset/vid_231_frame_116.jpg  
  inflating: YOLO_Dataset/vid_67_frame_312.jpg  
  inflating: YOLO_Dataset/vid_53_frame_134.jpg  
  inflating: YOLO_Dataset/vid_2_frame_29906.jpg  
  inflating: YOLO_Dataset/vid_2_frame_12588.jpg  
  inflating: YOLO_Dataset/vid_5_frame_1620.jpg  
  inflating: YOLO_Dataset/vid_38_frame_670.jpg  
  inflating: YOLO_Dataset/vid_28_frame_2424.jpg  
  inflating: YOLO_Dataset/vid_80_frame_97.jpg  
  inflating: YOLO_Dataset/vid_41_frame_458.jpg  
  inflating: YOLO_Dataset/vid_112_frame_101.jpg  
  inflating: YOLO_Dataset/vid_37_frame_114.jpg  
  inflating: 

  inflating: YOLO_Dataset/vid_3_frame_22780.jpg  
  inflating: YOLO_Dataset/vid_88_frame_283.jpg  
  inflating: YOLO_Dataset/vid_42_frame_892.jpg  
  inflating: YOLO_Dataset/vid_53_frame_297.jpg  
  inflating: YOLO_Dataset/vid_99_frame_227.jpg  
  inflating: YOLO_Dataset/vid_217_frame_19.jpg  
  inflating: YOLO_Dataset/vid_41_frame_185.jpg  
  inflating: YOLO_Dataset/vid_5_frame_1447.jpg  
  inflating: YOLO_Dataset/vid_30_frame_1973.jpg  
  inflating: YOLO_Dataset/vid_31_frame_2122.jpg  
  inflating: YOLO_Dataset/vid_38_frame_433.jpg  
  inflating: YOLO_Dataset/vid_50_frame_128.jpg  
  inflating: YOLO_Dataset/vid_31_frame_2102.jpg  
  inflating: YOLO_Dataset/vid_2_frame_44856.jpg  
  inflating: YOLO_Dataset/vid_31_frame_2485.jpg  
  inflating: YOLO_Dataset/vid_110_frame_167.jpg  
  inflating: YOLO_Dataset/vid_78_frame_344.jpg  
  inflating: YOLO_Dataset/vid_2_frame_25055.jpg  
  inflating: YOLO_Dataset/vid_28_frame_1214.jpg  
  inflating: YOLO_Dataset/vid_37_frame_119.jpg  
  inflating

  inflating: YOLO_Dataset/vid_31_frame_2519.jpg  
  inflating: YOLO_Dataset/vid_42_frame_300.jpg  
  inflating: YOLO_Dataset/vid_28_frame_1500.jpg  
  inflating: YOLO_Dataset/vid_40_frame_803.jpg  
  inflating: YOLO_Dataset/vid_97_frame_16.jpg  
  inflating: YOLO_Dataset/vid_98_frame_30.jpg  
  inflating: YOLO_Dataset/vid_18_frame_2007.jpg  
  inflating: YOLO_Dataset/vid_31_frame_1886.jpg  
  inflating: YOLO_Dataset/vid_38_frame_715.jpg  
  inflating: YOLO_Dataset/vid_38_frame_305.jpg  
  inflating: YOLO_Dataset/vid_18_frame_1067.jpg  
  inflating: YOLO_Dataset/vid_213_frame_182.jpg  
  inflating: YOLO_Dataset/vid_42_frame_363.jpg  
  inflating: YOLO_Dataset/vid_5_frame_2472.jpg  
  inflating: YOLO_Dataset/vid_18_frame_2319.jpg  
  inflating: YOLO_Dataset/vid_5_frame_2370.jpg  
  inflating: YOLO_Dataset/vid_76_frame_130.jpg  
  inflating: YOLO_Dataset/vid_41_frame_78.jpg  
  inflating: YOLO_Dataset/vid_28_frame_2062.jpg  
  inflating: YOLO_Dataset/vid_101_frame_145.jpg  
  inflating: Y

  inflating: YOLO_Dataset/vid_121_frame_11249.jpg  
  inflating: YOLO_Dataset/vid_31_frame_2330.jpg  
  inflating: YOLO_Dataset/vid_30_frame_1170.jpg  
  inflating: YOLO_Dataset/vid_30_frame_1382.jpg  
  inflating: YOLO_Dataset/vid_121_frame_9975.jpg  
  inflating: YOLO_Dataset/vid_45_frame_671.jpg  
  inflating: YOLO_Dataset/vid_2_frame_39667.jpg  
  inflating: YOLO_Dataset/vid_18_frame_1620.jpg  
  inflating: YOLO_Dataset/vid_31_frame_3055.jpg  
  inflating: YOLO_Dataset/vid_31_frame_2941.jpg  
  inflating: YOLO_Dataset/vid_40_frame_1332.jpg  
  inflating: YOLO_Dataset/vid_206_frame_48.jpg  
  inflating: YOLO_Dataset/vid_28_frame_1206.jpg  
  inflating: YOLO_Dataset/vid_42_frame_817.jpg  
  inflating: YOLO_Dataset/vid_201_frame_20.jpg  
  inflating: YOLO_Dataset/vid_31_frame_2659.jpg  
  inflating: YOLO_Dataset/vid_78_frame_122.jpg  
  inflating: YOLO_Dataset/vid_38_frame_74.jpg  
  inflating: YOLO_Dataset/vid_30_frame_2454.jpg  
  inflating: YOLO_Dataset/vid_38_frame_176.jpg  
  inf

## 2. Using Pretrained YOLOv3 Weights File to Start Training


First, import all the packages used in this tutorial:

In [3]:
import os
import random
import tempfile
import time
import multiprocessing
import subprocess
import math
import shutil
import math

from datetime import datetime
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

from models import Darknet
from utils.datasets import ImageLabelDataset
from utils.utils import model_info, print_args, Logger, visualize_and_save_to_local,xywh2xyxy
from yolo_tutorial_util import run_epoch
import validate
import warnings
import sys

##### section for all random seeds #####
torch.manual_seed(2)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
########################################

warnings.filterwarnings("ignore")
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

cuda = torch.cuda.is_available()
device = torch.device('cuda:0' if cuda else 'cpu')
num_cpu = multiprocessing.cpu_count() if cuda else 0

if cuda:
    torch.cuda.synchronize()
random.seed(0)
torch.manual_seed(0)

if cuda:
    torch.cuda.manual_seed(0)
    torch.cuda.manual_seed_all(0)
    torch.backends.cudnn.benchmark = True
    torch.cuda.empty_cache()

Successfully imported all packages and configured random seed to 17!

Training Config

In [5]:
# Training Related Config
batch_size = int(5)
optimizer_pick = "Adam"
model_cfg = "model_cfg/yolo_baseline.cfg"
weights_path = "sample-yolov3.weights"
output_path = "automatic"
dataset_path = "dataset/YOLO_Dataset/"
num_epochs = int(2) # Set them to 2048 during full dataset training
num_steps = 8388608
checkpoint_interval = int(1) # How often you want to get evaluation metric during training
val_tolerance = int(3)
min_epochs = int(3)

# Dataloader Related Config
data_aug = False # toggle for image augmentation
blur = False # Add blur to image
salt = False # Add "salt" noise to image
noise = False # Add noise to image
contrast = False # Add High Contrast to image
sharpen = False # Image Sharpen
ts = True # Tiling and Scaling
augment_affine = False # Affine
augment_hsv = False # HSV
lr_flip = False # left and right flip
ud_flip = False # up and down flip

# Training Hyperparameter Related Config
momentum = float(0.9)
gamma = float(0.95)
lr = float(0.001)
weight_decay = float(0.0)

xy_loss = float(2)
wh_loss= float(1.6)
no_object_loss = float(25)
object_loss = float(0.1)

# Debugging/Visualization Related Config
debug_mode = False
upload_dataset = False
vanilla_anchor = False
vis_batch = int(0)


Initializing Model

In [6]:
input_arguments = list(locals().items())

print("Initializing model")
model = Darknet(config_path=model_cfg,xy_loss=xy_loss,wh_loss=wh_loss,no_object_loss=no_object_loss,object_loss=object_loss,vanilla_anchor=vanilla_anchor)

Initializing model


Processing Training Config

In [7]:
if output_path == "automatic":
    current_month = datetime.now().strftime('%B').lower()
    current_year = str(datetime.now().year)
    if not os.path.exists(os.path.join('outputs/', current_month + '-' + current_year + '-experiments/' + model_cfg.split('.')[0].split('/')[-1])):
        os.makedirs(os.path.join('outputs/', current_month + '-' + current_year + '-experiments/' + model_cfg.split('.')[0].split('/')[-1]))
    output_uri = os.path.join('outputs/', current_month + '-' + current_year + '-experiments/' + model_cfg.split('.')[0].split('/')[-1])
else:
    output_uri = output_path

img_width, img_height = model.img_size()
bw  = model.get_bw()
validate_uri, train_uri = model.get_links()
num_validate_images, num_train_images = model.num_images()
conf_thresh, nms_thresh, iou_thresh = model.get_threshs()
num_classes = model.get_num_classes()
loss_constant = model.get_loss_constant()
conv_activation = model.get_conv_activation()
anchors = model.get_anchors()
onnx_name = model.get_onnx_name()

start_epoch = 0
weights_path = weights_path

Initialize Data Loaders

In [8]:
with tempfile.TemporaryDirectory() as tensorboard_data_dir:
    print("Initializing data loaders")
    train_data_loader = torch.utils.data.DataLoader(
        ImageLabelDataset(train_uri, dataset_path=dataset_path, width=img_width, height=img_height, augment_hsv=augment_hsv,
                            augment_affine=augment_affine, num_images=num_train_images,
                            bw=bw, n_cpu=num_cpu, lr_flip=lr_flip, ud_flip=ud_flip,vis_batch=vis_batch,data_aug=data_aug,blur=blur,salt=salt,noise=noise,contrast=contrast,sharpen=sharpen,ts=ts,debug_mode=debug_mode, upload_dataset=upload_dataset),
        batch_size=(1 if debug_mode else batch_size),
        shuffle=(False if debug_mode else True),
        num_workers=(0 if vis_batch else num_cpu),
        pin_memory=cuda)
    print("Num train images: ", len(train_data_loader.dataset))

    validate_data_loader = torch.utils.data.DataLoader(
        ImageLabelDataset(validate_uri, dataset_path=dataset_path, width=img_width, height=img_height, augment_hsv=False,
                            augment_affine=False, num_images=num_validate_images,
                            bw=bw, n_cpu=num_cpu, lr_flip=False, ud_flip=False,vis_batch=vis_batch,data_aug=False,blur=False,salt=False,noise=False,contrast=False,sharpen=False,ts=ts,debug_mode=debug_mode, upload_dataset=upload_dataset),
        batch_size=(1 if debug_mode else batch_size),
        shuffle=False,
        num_workers=(0 if vis_batch else num_cpu),
        pin_memory=cuda)

Initializing data loaders
Num train images:  44


Initialize Optimizer

In [9]:
if optimizer_pick == "Adam":
    print("Using Adam Optimizer")
    optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()),
                                lr=lr, weight_decay=weight_decay)
elif optimizer_pick == "SGD":
    print("Using SGD Optimizer")
    optimizer = torch.optim.SGD(filter(lambda p: p.requires_grad, model.parameters()),
                            lr=lr, momentum=momentum, weight_decay=weight_decay)
else:
    raise Exception(f"Invalid optimizer name: {optimizer_pick}")
print("Loading weights")
model.load_weights(weights_path, model.get_start_weight_dim())

# Set scheduler
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=gamma)

Using Adam Optimizer
Loading weights


Sending Model to GPUs if we are in GPU mode

### Let's Dance (Training)

In [10]:
if torch.cuda.device_count() > 1:
    print('Using ', torch.cuda.device_count(), ' GPUs')
    model = nn.DataParallel(model)
model = model.to(device, non_blocking=True)

val_loss = 999  # using a high number for validation loss
val_loss_counter = 0
step = [0]  # wrapping in an array so it is mutable
epoch = start_epoch
while epoch < num_epochs and step[0] < num_steps:
    epoch += 1
    scheduler.step()
    model.train()
    run_epoch(label_prefix="train", data_loader=train_data_loader, epoch=epoch,
                step=step, model=model, num_epochs=num_epochs, num_steps=num_steps,
                optimizer=optimizer, device=device)
    print('Completed epoch: ', epoch)
    # Update best loss
    if epoch % checkpoint_interval == 0 or epoch == num_epochs or step[0] >= num_steps:
        # First, save the weights
        save_weights_uri = os.path.join(output_uri, "{epoch}.weights".format(epoch=epoch))
        model.save_weights(save_weights_uri)

        with torch.no_grad():
            print("Calculating loss on validate data")
            epoch_losses, epoch_time_total, epoch_num_targets = run_epoch(
                label_prefix="validate", data_loader=validate_data_loader, epoch=epoch,
                model=model, num_epochs=num_epochs, num_steps=num_steps, optimizer=None,
                step=step, device=device)
            avg_epoch_loss = epoch_losses[0] / epoch_num_targets
            print('Average Validation Loss: {0:10.6f}'.format(avg_epoch_loss))

            if avg_epoch_loss > val_loss and epoch > min_epochs:
                val_loss_counter += 1
                print(f"Validation loss did not decrease for {val_loss_counter}"
                        f" consecutive check(s)")
            else:
                print("Validation loss decreased. Yay!!")
                val_loss_counter = 0
                val_loss = avg_epoch_loss
                ##### updating best result for optuna study #####
                result = open("logs/result.txt", "w" )
                result.write(str(avg_epoch_loss))
                result.close() 
                ###########################################
            validate.validate(dataloader=validate_data_loader, model=model, device=device, step=step[0], bbox_all=False,debug_mode=debug_mode)
            if val_loss_counter == val_tolerance:
                print("Validation loss stopped decreasing over the last " + str(val_tolerance) + " checkpoints, creating onnx file")
                with tempfile.NamedTemporaryFile() as tmpfile:
                    model.save_weights(tmpfile.name)
                    weights_name = tmpfile.name
                    cfg_name = os.path.join(tempfile.gettempdir(), model_cfg.split('/')[-1].split('.')[0] + '.tmp')
                    onnx_gen = subprocess.call(['python3', 'yolo2onnx.py', '--cfg_name', cfg_name, '--weights_name', weights_name])
                    save_weights_uri = os.path.join(output_uri, onnx_name)
                    os.rename(weights_name, save_weights_uri)
                    try:
                        os.remove(onnx_name)
                        os.remove(cfg_name)
                    except:
                        pass
                break

Model in train mode
train Epoch: 1, Batch: 1/9, Total:  20.893673,   L-x:  0.04%,   L-y:  0.05%,   L-w: 49.57%,   L-h: 49.62%,   L-noobj:  0.12%,   L-obj:  0.59%
train Epoch: 1, Batch: 2/9, Total:  13.387220,   L-x:  0.06%,   L-y:  0.08%,   L-w: 49.57%,   L-h: 49.13%,   L-noobj:  0.31%,   L-obj:  0.85%
train Epoch: 1, Batch: 3/9, Total:   0.187411,   L-x:  2.95%,   L-y:  2.73%,   L-w: 29.67%,   L-h: 25.30%,   L-noobj:  5.40%,   L-obj: 33.96%
train Epoch: 1, Batch: 4/9, Total:   0.191538,   L-x:  1.75%,   L-y:  1.70%,   L-w: 59.66%,   L-h: 10.55%,   L-noobj:  8.03%,   L-obj: 18.30%
train Epoch: 1, Batch: 5/9, Total: 211.820068,   L-x:  0.01%,   L-y:  0.01%,   L-w: 50.17%,   L-h: 49.67%,   L-noobj:  0.04%,   L-obj:  0.10%
train Epoch: 1, Batch: 6/9, Total:   5.021717,   L-x:  0.09%,   L-y:  0.09%,   L-w: 49.78%,   L-h: 48.86%,   L-noobj:  0.18%,   L-obj:  1.01%
train Epoch: 1, Batch: 7/9, Total:   3.519341,   L-x:  0.11%,   L-y:  0.10%,   L-w: 49.93%,   L-h: 48.96%,   L-noobj:  0.12%,   

Computing mAP:   0%|          | 0/3 [00:00<?, ?it/s]

validate Epoch: 1, Batch: 3/3, Total:   0.325124,   L-x:  8.01%,   L-y:  3.42%,   L-w: 41.70%,   L-h: 18.08%,   L-noobj: 14.24%,   L-obj: 14.54%
Average Validation Loss:   0.301875
Validation loss decreased. Yay!!
Calculating mAP - Model in evaluation mode


Computing mAP: 100%|██████████| 3/3 [00:02<00:00,  1.38it/s]

mAP: 29.85%, Recall: 64.80%, Precision: 12.85%
Model in train mode


train Epoch: 2, Batch: 1/9, Total:  18.192218,   L-x:  0.06%,   L-y:  0.07%,   L-w: 50.11%,   L-h: 49.08%,   L-noobj:  0.33%,   L-obj:  0.36%
train Epoch: 2, Batch: 2/9, Total:   6.179153,   L-x:  0.06%,   L-y:  0.04%,   L-w: 50.40%,   L-h: 48.98%,   L-noobj:  0.06%,   L-obj:  0.46%
train Epoch: 2, Batch: 3/9, Total:  73.278450,   L-x:  0.02%,   L-y:  0.01%,   L-w: 50.63%,   L-h: 49.14%,   L-noobj:  0.08%,   L-obj:  0.12%
train Epoch: 2, Batch: 4/9, Total:  26.738680,   L-x:  0.04%,   L-y:  0.02%,   L-w: 50.62%,   L-h: 49.04%,   L-noobj:  0.10%,   L-obj:  0.18%
train Epoch: 2, Batch: 5/9, Total:   2.906375,   L-x:  0.14%,   L-y:  0.12%,   L-w: 50.68%,   L-h: 48.09%,   L-noobj:  0.32%,   L-obj:  0.66%
train Epoch: 2, Batch: 6/9, Total:  22.959944,   L-x:  0.04%,   L-y:  0.05%,   L-w: 51.18%,   L-h: 48.27%,   L-noobj:  0.30%,   L-obj:  0.15%
train Epoch: 2, Batch: 7/9, Total:   0.068230,   L-x:  4.94%,   L-y:  5.74%,   L-w: 34.62%,   L-h: 22.21%,   L-noobj:  9.08%,   L-obj: 23.42%
train 

Computing mAP:   0%|          | 0/3 [00:00<?, ?it/s]

validate Epoch: 2, Batch: 3/3, Total:   0.360462,   L-x:  5.51%,   L-y:  4.63%,   L-w: 41.98%,   L-h: 22.66%,   L-noobj: 13.17%,   L-obj: 12.05%
Average Validation Loss:   0.177869
Validation loss decreased. Yay!!
Calculating mAP - Model in evaluation mode


Computing mAP: 100%|██████████| 3/3 [00:01<00:00,  1.53it/s]

mAP: 31.34%, Recall: 57.89%, Precision: 16.24%


## 3. Inference

Download target video file for inference

In [11]:
from IPython.display import Video

! wget https://storage.googleapis.com/mit-driverless-open-source/test_yolo_video.mp4

! ffmpeg -i test_yolo_video.mp4 test.mp4 && rm test_yolo_video.mp4

Video("test.mp4")

--2020-08-30 00:11:12--  https://storage.googleapis.com/mit-driverless-open-source/test_yolo_video.mp4
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.214.128, 172.217.212.128, 173.194.197.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.214.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3025082 (2.9M) [video/mp4]
Saving to: ‘test_yolo_video.mp4’

test_yolo_video.mp4 100%[===================>]   2.88M  --.-KB/s    in 0.03s   

2020-08-30 00:11:12 (107 MB/s) - ‘test_yolo_video.mp4’ saved [3025082/3025082]

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable

Download pretrained weights for inference

In [12]:
! wget https://storage.googleapis.com/mit-driverless-open-source/pretrained_yolo.weights

--2020-08-30 00:11:14--  https://storage.googleapis.com/mit-driverless-open-source/pretrained_yolo.weights
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.214.128, 172.217.212.128, 173.194.197.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.214.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘pretrained_yolo.weights’

pretrained_yolo.wei 100%[===================>] 236.52M  82.8MB/s    in 2.9s    

2020-08-30 00:11:17 (82.8 MB/s) - ‘pretrained_yolo.weights’ saved [248007048/248007048]



Import all packages for inference

In [13]:
import os
from os.path import isfile, join
import copy
import cv2
from tensorboardX import SummaryWriter
from PIL import Image, ImageDraw
import torchvision
from utils.nms import nms
from utils.utils import calculate_padding
from yolo_tutorial_util import single_img_detect, detect
from tqdm import tqdm

In [14]:
warnings.filterwarnings("ignore")
detection_tmp_path = "/tmp/detect/"

Set up config file for inference

In [15]:
target_path = "test.mp4"
output_path = "outputs/visualization/"
weights_path = "pretrained_yolo.weights"
conf_thres = float(0.8)
nms_thres = float(0.25)

In [18]:
cuda = torch.cuda.is_available()
device = torch.device('cuda:0' if cuda else 'cpu')
random.seed(0)
torch.manual_seed(0)
if cuda:
    torch.cuda.manual_seed(0)
    torch.cuda.manual_seed_all(0)
    torch.backends.cudnn.benchmark = True
    torch.cuda.empty_cache()
model = Darknet(config_path=model_cfg,xy_loss=xy_loss,wh_loss=wh_loss,no_object_loss=no_object_loss,object_loss=object_loss,vanilla_anchor=vanilla_anchor)

# Load weights
model.load_weights(weights_path, model.get_start_weight_dim())
model.to(device, non_blocking=True)

detect(target_path, output_path, model, device=device, conf_thres=conf_thres, nms_thres=nms_thres, detection_tmp_path=detection_tmp_path)

Detection Mode is: video


Doing Single Image Detection:   0%|          | 0/157 [00:00<?, ?it/s]

Frames per second using video.get(cv2.CAP_PROP_FPS) : 26.0


Creating Video: 100%|██████████| 157/157 [00:00<00:00, 201.51it/s]


In [19]:
! cd outputs/visualization/ && ffmpeg -i test.mp4 output.mp4 && rm test.mp4 && cd ../..

Video("outputs/visualization/output.mp4")

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

**Notice:** Again, you can further improve the accuracy of the cone detection network by switching YOLOv3 backbone to the most recent published YOLOv4

![](https://user-images.githubusercontent.com/22118253/70950893-e2de6980-202f-11ea-9a16-399579926ee5.gif)

Congratulations! You've finished all the content of this tutorial!
Hope you enjoy playing with the our object detection model. If you are interested,  please refer to our paper and GitHub Repo for further details.

## Reference
[1] Kieran Strobel, Sibo Zhu, Raphael Chang and Skanda Koppula.
**Accurate, Low-Latency Visual Perception for Autonomous Racing:Challenges, Mechanisms, and Practical Solutions**. In *IROS* 2020.
[[paper]](https://arxiv.org/abs/2007.13971), [[code]](https://github.com/cv-core/MIT-Driverless-CV-TrainingInfra).